In [1]:
import graphlab as gl
import scipy as sp
import numpy as np
import xgboost as xgb

from HAMMER import train_xgboost, xgboost_predict

[WARNING] Unable to write current GraphLab Create license to /home/alvas/.graphlab/config. Ensure that this user account has write permission to /home/alvas/.graphlab/config to save the license for offline use.
[INFO] This non-commercial license of GraphLab Create is assigned to liling@coli.uni-saarland.de and will expire on October 11, 2016. For commercial licensing options, visit https://dato.com/buy/.

[INFO] Start server at: ipc:///tmp/graphlab_server-4314 - Server binary: /usr/local/lib/python2.7/dist-packages/graphlab/unity_server - Server log: /tmp/graphlab_server_1454467908.log
[INFO] GraphLab Server Version: 1.8.1
[WARNING] Unable to create session in specified location: '/home/alvas/.graphlab/artifacts'. Using: '/var/tmp/graphlab-alvas/4314/tmp_session_0bfd3202-9c71-43dd-bf88-878fce463181'


In [2]:
sts_train = gl.SFrame('../sts2016_train.stasis/')
sts_test = gl.SFrame('../sts2016_test.stasis/')
sts_train = sts_train.dropna(columns=['Score'])

In [3]:
feat1 = ['prop_harmonic', 'DLS_compose_ppmi', 'DLS_compose_cbow']
feat2 = ['glove_cosine', 'prop_harmonic', 'DLS_compose_ppmi', 'DLS_compose_cbow', 'REVAL']
m1 = gl.boosted_trees_regression.create(sts_train, target='Score', features=feat1, validation_set=None)
m2 = gl.boosted_trees_regression.create(sts_train, target='Score', features=feat2, validation_set=None)



PROGRESS: Boosted trees regression:
PROGRESS: --------------------------------------------------------
PROGRESS: Number of examples          : 13597
PROGRESS: Number of features          : 3
PROGRESS: Number of unpacked features : 3
PROGRESS: +-----------+--------------+--------------------+---------------+
PROGRESS: | Iteration | Elapsed Time | Training-max_error | Training-rmse |
PROGRESS: +-----------+--------------+--------------------+---------------+
PROGRESS: | 1         | 1.033796     | 4.26247            | 2.12918       |
PROGRESS: | 2         | 1.041294     | 4.04426            | 1.65681       |
PROGRESS: | 3         | 1.048528     | 3.99651            | 1.36361       |
PROGRESS: | 4         | 1.055530     | 3.99096            | 1.18997       |
PROGRESS: | 5         | 1.063725     | 3.99484            | 1.09257       |
PROGRESS: | 6         | 1.073306     | 4.00021            | 1.03826       |
PROGRESS: | 7         | 1.080753     | 4.03883            | 1.0079        |
PROGRES

In [4]:
train_valid = sts_train['glove_cosine', 'prop_harmonic', 'DLS_compose_ppmi', 'DLS_compose_cbow', 'REVAL', 'BEER', 'Score'].to_dataframe()

In [5]:
gbm = train_xgboost(train_valid, feat2, num_boost_round=5000, eta=0.02, max_depth=8, test_size=0.2)

Will train until eval error hasn't decreased in 100 rounds.
[0]	train-rmse:0.896464	eval-rmse:0.901914
[1]	train-rmse:0.880858	eval-rmse:0.886520
[2]	train-rmse:0.865902	eval-rmse:0.871817
[3]	train-rmse:0.851279	eval-rmse:0.857429
[4]	train-rmse:0.836885	eval-rmse:0.843295
[5]	train-rmse:0.822560	eval-rmse:0.829205
[6]	train-rmse:0.808846	eval-rmse:0.815705
[7]	train-rmse:0.795200	eval-rmse:0.802237
[8]	train-rmse:0.781826	eval-rmse:0.789128
[9]	train-rmse:0.768741	eval-rmse:0.776310
[10]	train-rmse:0.755969	eval-rmse:0.763751
[11]	train-rmse:0.743701	eval-rmse:0.751778
[12]	train-rmse:0.731770	eval-rmse:0.740124
[13]	train-rmse:0.719787	eval-rmse:0.728460
[14]	train-rmse:0.708106	eval-rmse:0.717065
[15]	train-rmse:0.696701	eval-rmse:0.705939
[16]	train-rmse:0.685592	eval-rmse:0.695079
[17]	train-rmse:0.674940	eval-rmse:0.684842
[18]	train-rmse:0.664610	eval-rmse:0.674807
[19]	train-rmse:0.654474	eval-rmse:0.665032
[20]	train-rmse:0.644584	eval-rmse:0.655439
[21]	train-rmse:0.634838	e

Validating
RMSE: 1.017281


[506]	train-rmse:0.217146	eval-rmse:0.331781
[507]	train-rmse:0.217066	eval-rmse:0.331802
Stopping. Best iteration:
[407]	train-rmse:0.227002	eval-rmse:0.331622



In [6]:
predictions = xgboost_predict(gbm, sts_test[feat2].to_dataframe(), feat2)


In [7]:
import os, io

def output_to_file(team_name, run_name, target):
    test_dir = '../sts2016-english-v1.1/'
    filenames = [test_dir+i for i in os.listdir(test_dir) if i.endswith('ascii')]
    for infile in filenames:
        domain = infile.rpartition('.')[0].rpartition('.')[2]
        outfile = 'STS2016.OUTPUT.'+team_name+'.'+run_name+'.'+domain+'.txt'
        with io.open(infile, 'r') as fin, io.open(outfile, 'w') as fout:
            domain_data = sts_test.filter_by(domain, column_name='Domain')
            for line, row in zip(fin, domain_data):
                #line = line.split('\t')
                #s1, s2 = line[0], line[1]
                fout.write(unicode(row[target])+'\n')

sts_test.add_column(gl.SArray(m1.predict(sts_test)), name='pred_m1')
sts_test.add_column(gl.SArray(m2.predict(sts_test)), name='pred_m2')                
sts_test.add_column(gl.SArray(predictions), name='pred_xgboost')
output_to_file('wolvesaar', 'DLS-replica', 'pred_m1')
output_to_file('wolvesaar', 'lotsa-embeddings', 'pred_m2')
output_to_file('wolvesaar', 'xgboost', 'pred_xgboost')





